In [ ]:
package_paths = [
    "../input/pytorch-image-library/pytorch-image-models-master/pytorch-image-models-master",
]
import sys;

for pth in package_paths:
    sys.path.append(pth)

import timm

# Import

In [ ]:
import pandas as pd
import numpy as np
import cv2
import torch
import torch.nn as nn
import albumentations as A
import pytorch_lightning as pl
import matplotlib.pyplot as plt

from torch.utils.data import Dataset, DataLoader
from albumentations.core.composition import Compose, OneOf
from albumentations.pytorch import ToTensorV2

from pytorch_lightning import Trainer, seed_everything
from pytorch_lightning import Callback
from pytorch_lightning.loggers import CSVLogger
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.model_selection import StratifiedKFold

In [ ]:
print(f"PyTorch Lightning version: {pl.__version__}")

# Config

In [ ]:
DEBUG = False

class CFG:
    seed = 42
    model_name = 'tf_efficientnet_b5_ns'
    pretrained = True
    img_size = 512
    num_classes = 6
    lr = 1e-4
    max_lr = 1e-3
    pct_start = 0.3
    div_factor = 1.0e+3
    final_div_factor = 1.0e+3
    num_epochs = 5
    batch_size = 16
    accum = 1
    precision = 16
    n_fold = 5
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
PATH = "../input/plant-pathology-2021-fgvc8/"
TEST_DIR = PATH + 'test_images/'

In [ ]:
seed_everything(CFG.seed)

# Define Dataset

In [ ]:
class PlantDataset(Dataset):
    def __init__(self, df, transform=None):
        self.image_id = df['image'].values
        self.labels = df.iloc[:, 2:].values
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image_id = self.image_id[idx]
        label = torch.tensor(self.labels[idx].astype('int8'), dtype=torch.float32)
        
        image_path = TEST_DIR + image_id
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        augmented = self.transform(image=image)
        image = augmented['image']
        return {'image':image, 'target': label}

In [ ]:
def get_transform(phase: str):
    if phase == 'train':
        return Compose([
            A.RandomResizedCrop(height=CFG.img_size, width=CFG.img_size),
            A.Flip(p=0.5),
            A.RandomRotate90(p=0.5),
            A.ShiftScaleRotate(p=0.5),
            A.HueSaturationValue(p=0.5),
            A.OneOf([
                A.RandomBrightnessContrast(p=0.5),
                A.RandomGamma(p=0.5),
            ], p=0.5),
            A.OneOf([
                A.Blur(p=0.1),
                A.GaussianBlur(p=0.1),
                A.MotionBlur(p=0.1),
            ], p=0.1),
            A.OneOf([
                A.GaussNoise(p=0.1),
                A.ISONoise(p=0.1),
                A.GridDropout(ratio=0.5, p=0.2),
                A.CoarseDropout(max_holes=16, min_holes=8, max_height=16, max_width=16, min_height=8, min_width=8, p=0.2)
            ], p=0.2),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    else:
        return Compose([
            A.Resize(height=CFG.img_size, width=CFG.img_size),
            A.Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])

# Define Model

In [ ]:
class CustomModel(nn.Module):
    def __init__(self, model_name='tf_efficientnet_b0_ns', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        in_features = self.model.get_classifier().in_features
        fc = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(in_features, CFG.num_classes)
        )
        
        if 'efficientnet' in model_name:
            self.model.classifier = fc
        elif 'res' in model_name:
            self.model.fc = fc

    def forward(self, x):
        x = self.model(x)
        return x

## Inference

In [ ]:
checkpoint = "../input/pp-2021-efficientnet-model/last.ckpt"
model = CustomModel(model_name=CFG.model_name, pretrained=False)
model.load_state_dict(torch.load(checkpoint)['state_dict'])

In [ ]:
sub = pd.read_csv(PATH + "sample_submission.csv")
sub

In [ ]:
new_labels = ['healthy', 'scab', 'frog_eye_leaf_spot', 'complex', 'rust', 'powdery_mildew']

In [ ]:
tmp = pd.DataFrame(np.zeros([len(sub), len(new_labels)]), columns=new_labels)
sub = pd.concat([sub, tmp], axis=1)
sub

In [ ]:
test_dataset = PlantDataset(sub, get_transform('valid'))
test_loader = DataLoader(test_dataset, batch_size=CFG.batch_size, shuffle=False, num_workers=2)

In [ ]:
model.cuda()
model.eval()

sigmoid = nn.Sigmoid()

predictions = []
for batch in test_loader:
    image = batch['image'].cuda()
    with torch.no_grad():
        outputs = model(image)
        preds = outputs.detach().cpu()
        # The probability of 0.5 or more is considered positive.
        predictions.append(sigmoid(preds).numpy() > 0.55)

In [ ]:
predictions = pd.DataFrame(np.concatenate(predictions).astype(np.int), columns=new_labels)

In [ ]:
sub.iloc[:, 2:] = predictions
sub

In [ ]:
labels = []
for i, row in sub.iloc[:, 2:].iterrows():
    if ((row['healthy'] == 1) or row.sum() == 0):
        tmp = 'healthy'
    else:
        tmp = ' '.join(np.array(new_labels)[row==row.max()])
    labels.append(tmp)

In [ ]:
sub['labels'] = labels
sub[['image', 'labels']].to_csv('submission.csv', index=False)
sub